## # Preprocessing.ipynb

### # from Raw to Preprocessed_for_classification

In [1]:
import sys
import os
import os.path as osp
import re
from glob import glob
import shutil

import numpy as np
import pandas as pd

from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [2]:
base_PATH='/home/test/hdd/Dataset/chicken/Dataset/raw/classification'
# 전체 데이터 경로 가져오기
total_path=glob(osp.join(base_PATH, '*', '*.jpg'))

print('전체 데이터 수:',len(total_path))

전체 데이터 수: 30578


In [3]:
# 정상 닭 데이터 경로
class_0_path=glob(osp.join(base_PATH, '0','*.jpg'))
# 파계 데이터 경로
class_1_path=glob(osp.join(base_PATH, '1', '*.jpg'))

In [4]:
# 정상 닭 이미지 개수
print('정상 닭 이미지 수:',len(class_0_path))
# 파계 이미지 개수
print('파계 이미지 수:',len(class_1_path))

정상 닭 이미지 수: 30493
파계 이미지 수: 85


In [5]:
# 파계를 train : val : test = 55 : 15: 15 = 약 0.65 : 0.175 : 0.175 비율로 나눔
idx_class_1=list(range(0,len(class_1_path)))
np.random.seed(42)
np.random.shuffle(idx_class_1)

train_class_1=class_1_path[:55]
val_class_1=class_1_path[55:70]
test_class_1=class_1_path[70:]

In [6]:
# validation & test -> Normal : Abnormal = 90 : 10 가 되도록 구성
idx_class_0=list(range(0, len(class_0_path)))
np.random.seed(42)
np.random.shuffle(idx_class_0)

val_idx=15*9
test_idx=15*9

train_class_0=class_0_path[(val_idx+test_idx):]
val_class_0=class_0_path[:val_idx]
test_class_0=class_0_path[val_idx:(val_idx+test_idx)]

In [7]:
# 정상 닭과 파계 데이터를 합침
#train_path=train_class_0+train_class_1 + train_class_1*19  # over-sampling : 약 20배
train_path=train_class_0+train_class_1 ## 이번 연구에서는 over-sampling을 진행하지 않음
val_path=val_class_0+val_class_1
test_path=test_class_0+test_class_1

In [8]:
print(f"Train set size: {len(train_path)}")
## Oversampling 수행 시 31323, 미 수행시 30278개
print(f"Validation set size: {len(val_path)}")
print(f"Test set size: {len(test_path)}")

Train set size: 30278
Validation set size: 150
Test set size: 150


In [9]:
print('Oversampling', len(train_class_0+train_class_1 + train_class_1*19))
print('No Oversampling', len(train_class_0+train_class_1))
print('gap',len(train_class_0+train_class_1 + train_class_1*19)-len(train_class_0+train_class_1))

Oversampling 31323
No Oversampling 30278
gap 1045


In [ ]:
# train, val, test 폴더 생성
folder_list=['train','val','test']
dst_base_PATH='/home/test/hdd/Dataset/chicken/Dataset/preprocessed2'

for fd in folder_list:
    folder_path=osp.join(dst_base_PATH, fd)
    if not osp.exists(folder_path):
        os.mkdir(folder_path)
        
    for i in range(2):
        folder=osp.join(folder_path, str(i))
        if not osp.exists(folder):
            os.mkdir(folder)

In [25]:
dict_={'train': train_path, 'val': val_path, 'test': test_path}

# preprocessed 폴더로 데이터 복사
for key, values in dict_.items():
    for path in tqdm(values):
        img_src=path
        json_src=path.replace('.jpg','.json')
        
        label=path.split('/')[-2]
        img_name=osp.split(img_src)[1]
        json_name=osp.split(json_src)[1]
                
        img_dst=osp.join(dst_base_PATH, key, label, img_name)
        json_dst=osp.join(dst_base_PATH, key, label, json_name)
        
        # for oversampling
        if key=='train':
            if osp.exists(img_dst):
                while True:
                    img_name='a'+img_name
                    img_dst=osp.join(dst_base_PATH, key, label, img_name)
                    if not osp.exists(img_dst):
                        json_dst=img_dst.replace('.jpg','.json')
                        break
                    
        
        _=shutil.copyfile(img_src, img_dst)
        _=shutil.copyfile(json_src, json_dst)

100%|██████████| 150/150 [00:00<00:00, 416.36it/s]
